In [1]:
import torch
import numpy as np

## Tensor

In [26]:
# 创建tensor
n = np.ones((3, 4))

t = torch.ones(3, 4)
t = torch.rand(5, 3)
t = torch.zeros(3, 4, dtype=torch.long)

a = torch.tensor([1, 2, 3])
a = torch.randn_like(a, dtype=torch.float)

y = t.new_empty(2, 3)       # 复用t的其他属性

print("Numpy: ", n)
print("Tensor: ", t)
print(a)
print(y)

Numpy:  [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
Tensor:  tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])
tensor([-1.9611, -0.2047,  0.6853])
tensor([[7959390389040738153, 2318285298082652788, 8675445202132104482],
        [7957695011165139568, 2318365875964093043, 7233184988217307170]])


In [27]:
# 基础属性
print(t.size())
print(t.shape)

torch.Size([3, 4])
torch.Size([3, 4])


In [29]:
# 简单运算
x = torch.rand(2, 3)
y = torch.rand_like(x)

print(x + y)
print(torch.add(x, y))

tensor([[1.1634, 1.8894, 1.0713],
        [0.5683, 1.0986, 0.8609]])
tensor([[1.1634, 1.8894, 1.0713],
        [0.5683, 1.0986, 0.8609]])


In [30]:
# in-place运算 会改变变量的值
y.add_(x)
print(y)

tensor([[1.1634, 1.8894, 1.0713],
        [0.5683, 1.0986, 0.8609]])


In [31]:
# 切片
print(x[:, 1])

tensor([0.9541, 0.4638])


In [32]:
# resize
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 2, 4)

print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 2, 4])


In [33]:
# get value(if only have one element)
x = torch.rand(1)
print(x)
print(x.item())

tensor([0.1197])
0.11974674463272095


### Tensor与Numpy转换

In [39]:
# torch -> numpy
t = torch.rand(2, 3)
n = t.numpy()

print(t)
print(n)

tensor([[0.2107, 0.2437, 0.0706],
        [0.6233, 0.4380, 0.4589]])
[[0.21070534 0.24373996 0.07062978]
 [0.6232684  0.43795878 0.45893037]]


In [37]:
# numpy -> tensor
n = np.random.rand(3, 2)
t = torch.from_numpy(n)

print(n)
print(t)

[[0.16070166 0.54871463]
 [0.07759188 0.32236617]
 [0.14265208 0.4026539 ]]
tensor([[0.1607, 0.5487],
        [0.0776, 0.3224],
        [0.1427, 0.4027]], dtype=torch.float64)


In [40]:
# 修改一个两个都会跟着改的
t.add_(1)

print(t)
print(n)

tensor([[1.2107, 1.2437, 1.0706],
        [1.6233, 1.4380, 1.4589]])
[[1.2107053 1.24374   1.0706298]
 [1.6232684 1.4379587 1.4589304]]


In [6]:
# 复制之后就不会共享内存了
t = torch.rand(2, 3)
n = np.random.rand(2, 3)

# 二者的函数名是不一样的
t_ = n.copy()
n_ = t.clone()

t.zero_()
n = np.zeros((2, 3))

print(t_)
print(n_)

[[0.30370266 0.36835002 0.99111293]
 [0.36966819 0.68330543 0.12247895]]
tensor([[0.5291, 0.4583, 0.6517],
        [0.5869, 0.5024, 0.0601]])


In [57]:
# 如果添加了求导，则需要将data转换为numpy
n = torch.rand(2, 3, requires_grad=True)
t = n.data.numpy()

### CUDA

In [46]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.rand(2, 3, device=device)
    x = torch.rand(2, 3)
    x = x.to(device)

    z = x + y
    print(z)
    print(z.to("cpu"))

tensor([[1.3248, 0.3975, 0.7192],
        [0.7229, 0.7046, 0.7572]], device='cuda:0')
tensor([[1.3248, 0.3975, 0.7192],
        [0.7229, 0.7046, 0.7572]])


## 自动求导

In [50]:
x = torch.randn(4, 1, requires_grad=True)
b = torch.randn(4, 1, requires_grad=True)
W = torch.randn(4, 4)

# y = torch.mm(torch.mm(torch.t(x), W), b)
y = x.t().mm(W).mm(b)
print(y)

tensor([[1.7932]], grad_fn=<MmBackward>)


In [51]:
print(x.grad)

y.backward()

print(x.grad)

None
tensor([[ 0.4634],
        [-1.1022],
        [-3.5316],
        [ 1.3660]])


In [24]:
x = torch.randn(4, 1, requires_grad=True)
y = torch.mm(torch.t(x), x)
print(x)
print(y)

y.backward(retain_graph=True)   # 可以再次求导，否则只能backward一次
print(x.grad)

tensor([[-0.7865],
        [-0.1405],
        [-0.2294],
        [ 0.3251]], requires_grad=True)
tensor([[0.7966]], grad_fn=<MmBackward>)
tensor([[-1.5730],
        [-0.2811],
        [-0.4588],
        [ 0.6501]])


In [25]:
print(x)
print(y)

y.backward(retain_graph=True)
print(x.grad)

tensor([[-0.7865],
        [-0.1405],
        [-0.2294],
        [ 0.3251]], requires_grad=True)
tensor([[0.7966]], grad_fn=<MmBackward>)
tensor([[-3.1460],
        [-0.5622],
        [-0.9177],
        [ 1.3003]])


In [26]:
x.grad.zero_()  # 梯度清零
y.backward()
print(x.grad)   # 跟第一次的值相同

tensor([[-1.5730],
        [-0.2811],
        [-0.4588],
        [ 0.6501]])


## 全连接

In [5]:
input = torch.randn(10, 100)    # 第一个10是batch size
linear_network = torch.nn.Linear(100, 200)
output = linear_network(input)

print(output.shape)

for name, parameter in linear_network.named_parameters():
    print(name, ':', parameter.size())

torch.Size([10, 200])
weight : torch.Size([200, 100])
bias : torch.Size([200])
